<a href="https://colab.research.google.com/github/gmagannaDevelop/segnet/blob/mru_tests/example_notebooks/MultiResUNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MultiResUNet
## DCI-Net
### Primera Implementación
#### Gustavo Magaña

\########################################################

In [1]:
!ls

drive  sample_data


In [2]:
## If changes are made to the repo, uninstall and
## a fresh pip install are required, so it seems.
#!pip uninstall segnet
!pip install git+https://github.com/gmagannaDevelop/segnet.git@mru_tests

  Cloning https://github.com/gmagannaDevelop/segnet.git (to revision mru_tests) to /tmp/pip-req-build-l0e6bgvi
  Running command git clone -q https://github.com/gmagannaDevelop/segnet.git /tmp/pip-req-build-l0e6bgvi
  Running command git checkout -b mru_tests --track origin/mru_tests
  Switched to a new branch 'mru_tests'
  Branch 'mru_tests' set up to track remote branch 'mru_tests' from 'origin'.
  Created wheel for segnet: filename=segnet-0.4-cp36-none-any.whl size=22786 sha256=c7099eb437cdf0d152d20d3d5b94a9027f079467f24d96fb05df325ca3ac90a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-n4vn7l85/wheels/f1/c1/51/0ee27f445c3a618413714a84d4f5500547456389437a85a87f
Successfully built segnet


In [3]:
### System-related
import sys
import os
#import importlib.util
###############################################################

### Machine learning specific
#import segmentation_models as sm
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
###############################################################

### In-Out
from skimage import io
import glob
###############################################################

### Numerical
import numpy as np
###############################################################

### Repo-specific (segnet)
from segnet.train import train_segnet
from segnet.models import multiresunet2 as mru
#from segnet.segnet.train.train_segnet import train_segnet
#import segnet.segnet as segnet

### Data-related
from google.colab import drive
drive.mount('/content/drive/')
###############################################################

Using TensorFlow backend.


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
!ls "drive/My Drive/DCI-Net/"

Checkpoints  Data		 multiresunet.py  Snippets
Colab_data   MultiResUNet.ipynb  __pycache__	  timing.py


In [4]:
!ls "drive/My Drive/DCI-Net/Colab_data/Dataset 1"

images	masks


In [9]:
# Define some example hyperparameters
batch_size = 8
epochs = 50
steps_per_epoch=100

# Declare the paths to use (following the Keras convention)
# https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator
root_dir   = "drive/My Drive/DCI-Net/"
data_path  = "drive/My Drive/DCI-Net/Colab_data/Dataset 1"
img_path   = os.path.join(data_path, 'images')
masks_path = os.path.join(data_path, 'masks')
model_file = os.path.join(root_dir, 'first_mutlires.h5')

# Instantiate a MultiResUNet model :
model = mru.MultiResUNet()

# Train the model!
history = train_segnet.train_segnet(
    model,
    img_path,
    masks_path,
    batch_size=batch_size,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    model_file=model_file,
)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/50
100/100 [==============================] - 3s 34ms/step - loss: 0.0000e+00 - jaccard_index: 1.0000 - dice_coef: 1.0000

Epoch 00001: val_jaccard_index improved from -inf to 1.00000, saving model to drive/My Drive/DCI-Net/first_mutlires.h5


/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/callbacks.py:676: RuntimeWarning: invalid value encountered in double_scalars
  logs[k] = self.totals[k] / self.seen


100/100 [==============================] - 77s 766ms/step - loss: nan - jaccard_index: 1.0000 - dice_coef: 1.0000 - val_loss: 0.0000e+00 - val_jaccard_index: 1.0000 - val_dice_coef: 1.0000
Epoch 2/50
100/100 [==============================] - 1s 10ms/step - loss: 0.0000e+00 - jaccard_index: 1.0000 - dice_coef: 1.0000

Epoch 00002: val_jaccard_index did not improve from 1.00000
100/100 [==============================] - 7s 67ms/step - loss: nan - jaccard_index: 1.0000 - dice_coef: 1.0000 - val_loss: 0.0000e+00 - val_jaccard_index: 1.0000 - val_dice_coef: 1.0000
Epoch 3/50
100/100 [==============================] - 1s 10ms/step - loss: 0.0000e+00 - jaccard_index: 1.0000 - dice_coef: 1.0000

Epoch 00003: val_jaccard_index did not improve from 1.00000
100/100 [==============================] - 6s 64ms/step - loss: nan - jaccard_index: 1.0000 - dice_coef: 1.0000 - val_loss: 0.0000e+00 - val_jaccard_index: 1.0000 - val_dice_coef: 1.0000
Epoch 4/50
100/100 [==============================] - 1

In [0]:
!cp time_logs.jsonl drive/My\ Drive/DCI-Net/

In [12]:
locals()[]

{'In': ['',
  "get_ipython().system('ls')",
  "get_ipython().system('pip install git+https://github.com/gmagannaDevelop/segnet.git@mru_tests')",
  "import sys\nimport os\n#import importlib.util\n###############################################################\n\n### Machine learning specific\n#import segmentation_models as sm\nfrom keras.optimizers import SGD\nfrom keras.callbacks import ModelCheckpoint, ReduceLROnPlateau\nfrom sklearn.model_selection import train_test_split\n###############################################################\n\n### In-Out\nfrom skimage import io\nimport glob\n###############################################################\n\n### Numerical\nimport numpy as np\n###############################################################\n\n### Repo-specific (segnet)\nfrom segnet.train import train_segnet\nfrom segnet.models import multiresunet2 as mru\n#from segnet.segnet.train.train_segnet import train_segnet\n#import segnet.segnet as segnet\n\n### Data-related\nfrom go